### ML Modelling
This notebook walks through the process of developing ML models both in sk-learn and in sasviya.ml

In [ ]:
import pandas as pd 
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
import random 
import numpy as np
import pickle

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier as GradientBoostingSK
from sasviya.ml.tree import ForestClassifier, GradientBoostingClassifier
import matplotlib.pyplot as plt
import os

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
data_path = '../../Data/cleaned_data/train_valid_test.csv'
numerical_features = ['CreditLineAge','DebtIncRatio','FICOScore','Inquiries6Mnths',
                      'LogAnnualInc','RevBalance','RevUtilization','Installment','InterestRate']
categorical_features = ['CreditPolicy','Delinquencies2Yrs','PublicRecord','Purpose']
features = numerical_features+categorical_features
target = 'Default'

In [ ]:
all_data = pd.read_csv(data_path)

train = all_data[all_data['_PartInd_']==1].reset_index(drop=True)
valid = all_data[all_data['_PartInd_']==2].reset_index(drop=True)
test = all_data[all_data['_PartInd_']==3].reset_index(drop=True)

train_defaults = train[target].sum()
valid_defaults = valid[target].sum()
test_defaults = test[target].sum()
print('Train Size:', train.shape[0], f'--- {target} Frequency:', f'{round(100*train_defaults/train.shape[0],2)}%')
print('Valid Size:', valid.shape[0], f'--- {target} Frequency:', f'{round(100*valid_defaults/valid.shape[0],2)}%')
print('Test Size:', test.shape[0], f'--- {target} Frequency:', f'{round(100*test_defaults/test.shape[0],2)}%')

In [ ]:
### SK-Learn Pipeline
num_pipeline = Pipeline([
    ("standard_scaler", StandardScaler())
])

cat_pipeline = Pipeline([
    ("ordinal_encoder", OrdinalEncoder()),
])

sk_pipeline = Pipeline([
    ("data_prep", ColumnTransformer([("num_pipeline", num_pipeline, numerical_features),("cat_pipeline", cat_pipeline, categorical_features)])),
    ('Forest_Model', RandomForestClassifier(random_state=42))])

sk_pipeline

In [ ]:
#fit the pipeline
sk_pipeline.fit(train[features], train[target])
#Save predictions 
train_pred_sk = sk_pipeline.predict(train[features])
valid_pred_sk = sk_pipeline.predict(valid[features])
test_pred_sk = sk_pipeline.predict(test[features])
#Compute Fit Metrics
train_f1_sk = f1_score(train[target], train_pred_sk)
valid_f1_sk = f1_score(valid[target], valid_pred_sk)
test_f1_sk = f1_score(test[target], test_pred_sk)

In [ ]:
viya_pipeline = Pipeline([
    ("data_prep", ColumnTransformer([("num_pipeline", num_pipeline, numerical_features),("cat_pipeline", cat_pipeline, categorical_features)])),
    ('Forest_Model', ForestClassifier(random_state=42))])

viya_pipeline

In [ ]:
#fit the pipeline
viya_pipeline.fit(train[features], train[target])
#Save predictions 
train_pred_viya = viya_pipeline.predict(train[features])
valid_pred_viya = viya_pipeline.predict(valid[features])
test_pred_viya = viya_pipeline.predict(test[features])
#Compute Fit Metrics
train_f1_viya = f1_score(train[target], train_pred_viya)
valid_f1_viya = f1_score(valid[target], valid_pred_viya)
test_f1_viya = f1_score(test[target], test_pred_viya)

In [ ]:
model_comparison = pd.DataFrame(
    {'Forest-SK': [train_f1_sk, valid_f1_sk, test_f1_sk], 
     'Forest-Viya': [train_f1_viya, valid_f1_viya, test_f1_viya]},
    ['Train F1', 'Valid F1', 'Test F1'])
100*model_comparison.round(4)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(20,5))
disp = ConfusionMatrixDisplay(confusion_matrix(test[target], test_pred_sk, normalize='pred'))
disp.plot(cmap=plt.cm.Blues, ax=axs[0])
axs[0].set_title('Test Confusion Matrix - SK')

disp = ConfusionMatrixDisplay(confusion_matrix(test[target], test_pred_viya, normalize='pred'))
disp.plot(cmap=plt.cm.Blues, ax=axs[1])
axs[1].set_title('Test Confusion Matrix - Viya')

plt.show()

### Your Task

Develop a Gradient Boosting Model, test different hyperparameters and compare its performance against the above trained Logistic and Forest Models.

![GB Classifier Overview](../../img/GB_Details_Python.png)

For further guidance: https://go.documentation.sas.com/doc/en/workbenchcdc/v_001/explore/n1kiea90s0276wn1xr0ig0hvkix6.htm

In [ ]:
# Instantiate the model, fit it and evaluate it
your_gb_model = 

In [ ]:
# Make sure to compute the Train F1, Valid F1 and Test F1
model_comparison = pd.DataFrame(
    {'Forest-SK': [train_f1_sk, valid_f1_sk, test_f1_sk], 
     'Forest-Viya': [train_f1_viya, valid_f1_viya, test_f1_viya],
     'GB': [_________]},
    ['Train F1', 'Valid F1', 'Test F1'])
100*model_comparison.round(4)

In [ ]:
# Produce confusion matrices to compare performance against the Logistic Regression and the Gradient Boosting
fig, axs = plt.subplots(ncols=3, figsize=(20,5))
disp = ConfusionMatrixDisplay(confusion_matrix(test[target], test_pred_sk, normalize='true'))
disp.plot(cmap=plt.cm.Blues, ax=axs[0])
axs[0].set_title('Test Confusion Matrix - Logistic')

disp = ConfusionMatrixDisplay(confusion_matrix(test[target], test_pred_viya, normalize='true'))
disp.plot(cmap=plt.cm.Blues, ax=axs[1])
axs[1].set_title('Test Confusion Matrix - Forest')

disp = ConfusionMatrixDisplay(confusion_matrix(test[target], _______, normalize='true'))
disp.plot(cmap=plt.cm.Blues, ax=axs[2])
axs[2].set_title('Test Confusion Matrix - GB')

plt.show()

#### Let's fit only the models for registration

In [ ]:
data_prep = Pipeline([("data_prep", ColumnTransformer([("num_pipeline", num_pipeline, numerical_features),("cat_pipeline", cat_pipeline, categorical_features)]))])
data_prep.fit(train[features])
x_train_prep = pd.DataFrame(data_prep.transform(train[features]), columns=features)
x_valid_prep = pd.DataFrame(data_prep.transform(valid[features]), columns=features)

In [ ]:
gb_viya_model = GradientBoostingClassifier(random_state=42)
gb_viya_model.fit(x_train_prep, train[target])

In [ ]:
sk_model = GradientBoostingSK(random_state=42)
sk_model.fit(x_train_prep, train[target])

### Saving Models
We create an artifacts folder and save the models

In [ ]:
mypath = 'artifacts'
if not os.path.isdir(mypath):
   os.makedirs(mypath)

with open('artifacts/sklearn_forest_pipeline.pkl', 'wb') as file:
    pickle.dump(sk_pipeline, file)

with open('artifacts/viya_forest_pipeline.pkl', 'wb') as file:
    pickle.dump(viya_pipeline, file)

with open('artifacts/data_prep.pkl', 'wb') as file:
    pickle.dump(data_prep, file)

with open('artifacts/sk_model.pkl', 'wb') as file:
    pickle.dump(sk_model, file)

gb_viya_model.save('artifacts/gb_model.pkl')